In [1]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import glob,os
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
from datetime import timedelta
from datetime import datetime
import tushare as ts
pro = ts.pro_api()

In [2]:
#导入股票交易的文件
path_1 = r'C:\Anaconda\trade_data\stock_trade'
file_1 = glob.glob(os.path.join(path_1, "TRD_Dalyr**.csv"))
file_list_1 = []
for i in file_1:
    fd_file = pd.read_csv(i, index_col=None)
    #剔除B股、科创板；剔除ST股
    fd_file = fd_file.query('Trdsta == 1').query('(Markettype == 1) | (Markettype == 4) | (Markettype == 16)')
    file_list_1.append(fd_file)
fd_stock_trade = pd.concat(file_list_1)

In [3]:
#导入股票指标的文件
path_2 = r'C:\Anaconda\trade_data\stock_ysindex'
file_2 = glob.glob(os.path.join(path_2, 'STK_MKT_DALYR**.csv'))
fd_stock_pb = pd.DataFrame()
for i in file_2:
    fd_file = pd.read_csv(i,index_col=None)
    fd_stock_pb = fd_stock_pb.append(fd_file)
#columns重命名
fd_stock_pb.rename(columns={'Symbol':'Stkcd','TradingDate':'Trddt'},inplace=True)

In [4]:
#合并数据
fd_info = pd.merge(fd_stock_trade,fd_stock_pb,on=['Stkcd','Trddt'],how = 'inner' ) #取交集
fd_info = fd_info.sort_values(['Stkcd','Trddt'])
fd_info['Trddt'] = pd.to_datetime(fd_info['Trddt'])

In [5]:
#对数据进行进一步筛选（次新股、退市公司、缺失值、重复值）
basic_data = pro.stock_basic(exchange='', fields='ts_code,symbol,list_status,list_date')
basic_data['list_date'] = pd.to_datetime(basic_data['list_date'])
basic_data = basic_data.rename(columns={'symbol':'Stkcd'})
basic_data['Stkcd'] = basic_data['Stkcd'].astype('int')
basic_data.drop_duplicates('Stkcd')
fd_new_info = pd.merge(fd_info,basic_data,on = 'Stkcd') #多对一合并
#剔除次新股
fd_new_info['delta'] = fd_new_info['Trddt'] - fd_new_info['list_date']
fd_new_info = fd_new_info[fd_new_info['delta'] > timedelta(120)]
#剔除退市公司
fd_new_info = fd_new_info.query('(list_status == "L")')
#剔除缺失值、重复值
fd_new_info.dropna()
fd_new_info.drop_duplicates(['Stkcd','Trddt'])
fd_new_info = fd_new_info.set_index('Trddt',drop=False)

In [ ]:
#将数据转换为月度数据，并剔除每月交易少于14天的股票
#保留每个月第一天的交易数据（调仓）
fd_trade_month = fd_new_info.groupby(fd_new_info['Stkcd']).resample('M').first()
#剔除每月交易日少于14天的数据
fd_trade_month['count'] = fd_new_info['Trdsta'].groupby(fd_new_info['Stkcd']).resample('M').sum()
fd_trade_month = fd_trade_month[fd_trade_month['count'] > 14]
#计算每个月的收益
fd_new_info['Dretnd'] = fd_new_info['Dretnd'] + 1
fd_trade_month['month_return'] = fd_new_info['Dretnd'].groupby(fd_new_info['Stkcd']).resample('M').prod()
#计算roa
fd_trade_month['ROA'] = fd_trade_month['PB']/fd_trade_month['PE']
fd_trade_month['BM'] = 1/fd_trade_month['PB']
fd_trade_month['EP'] = 1/fd_trade_month['PE']
#生成后续可以使用的月度交易数据
fd_trade_month = fd_trade_month[['Stkcd','Dsmvosd','BM','EP','ROA','month_return','ts_code']]
fd_trade_month.rename(columns={'Stkcd':"stock_id",'Dsmvosd':'MarketValue'},inplace=True)
fd_trade_month

In [ ]:
#导入市场因子、动量因子、投资因子(直接从CSMAR导入)
fd_five = pd.read_csv('C:/Anaconda/trade_data/five_factor/STK_MKT_FIVEFACMONTH.csv', index_col = None)
fd_five = fd_five[fd_five['Portfolios']==1]
fd_five['TradingMonth'] = pd.to_datetime(fd_five['TradingMonth'])
fd_five.rename(columns = {'TradingMonth':'date'},inplace = True)
fd_five = fd_five.set_index('date')
fd_five = fd_five[datetime(2005,1,1):datetime(2019,12,1)]
fd_five = fd_five.resample('M').last() #index转换为最后一天方便匹配

fd_dl = pd.read_csv('C:/Anaconda/trade_data/five_factor/STK_MKT_CARHARTFOURFACTORS.csv', index_col = None)
fd_dl['TradingMonth'] = pd.to_datetime(fd_dl['TradingMonth'])
fd_dl.rename(columns = {'TradingMonth':'date'},inplace = True)
fd_dl = fd_dl.set_index('date')
fd_dl = fd_dl[datetime(2005,1,1):datetime(2019,12,1)]
fd_dl = fd_dl.resample('M').last() #index转换为最后一天方便匹配

fd_dl_inv = pd.merge(fd_five,fd_dl,how='inner',on='date')
fd_dl_inv = fd_dl_inv[['RiskPremium1','CMA1','UMD1']]
fd_dl_inv

In [ ]:
#定义计算Liu ，Stambaugh and Yuan（2019）三因子，并使用函数封装
def liu3(df_daily):
    df_daily['label_sb3'] = pd.qcut(df_daily['MarketValue'],2,['small','large'])
    df_daily['label_ep3'] = pd.qcut(df_daily['EP'],[0, 0.3, 0.7, 1.0],['low','mid','high'])
    
    #市值和EP双重独立排序
    small_low3 = df_daily.query('(label_sb3 == "small") & (label_ep3 == "low")')
    small_mid3 = df_daily.query('(label_sb3 == "small") & (label_ep3 == "mid")')
    small_high3 = df_daily.query('(label_sb3 == "small") & (label_ep3 == "high")')
    
    large_low3 = df_daily.query('(label_sb3 == "large") & (label_ep3 == "low")')
    large_mid3 = df_daily.query('(label_sb3 == "large") & (label_ep3 == "mid")')
    large_high3 = df_daily.query('(label_sb3 == "large") & (label_ep3 == "high")')
    
    r_sl3 = (small_low3['month_return']*small_low3['MarketValue']).sum()/small_low3['MarketValue'].sum()
    r_sm3 = (small_mid3['month_return']*small_mid3['MarketValue']).sum()/small_mid3['MarketValue'].sum()
    r_sh3 = (small_high3['month_return']*small_high3['MarketValue']).sum()/small_high3['MarketValue'].sum()
    
    r_ll3 = (large_low3['month_return']*large_low3['MarketValue']).sum()/large_low3['MarketValue'].sum()
    r_lm3 = (large_mid3['month_return']*large_mid3['MarketValue']).sum()/large_mid3['MarketValue'].sum()
    r_lh3 = (large_high3['month_return']*large_high3['MarketValue']).sum()/large_high3['MarketValue'].sum()
    
    liu3_smb = (r_sl3+r_sm3+r_sh3-r_ll3-r_lm3-r_lh3)/3
    liu3_hml = (r_lh3+r_sh3-r_ll3-r_sl3)/2
    
    return liu3_smb,liu3_hml

In [ ]:
#定义计算Liu，Shi and Lian（2019）四因子，并使用函数封装
def liu4(df_daily):
    df_daily['label_sb4'] = pd.qcut(df_daily['MarketValue'],2,['small','large'])
    df_daily['label_bm4'] = pd.qcut(df_daily['BM'],[0, 0.3, 0.7, 1.0],['low','mid','high'])
    df_daily['label_roa4'] = pd.qcut(df_daily['ROA'],[0, 0.3, 0.7, 1.0],['rlow','rmid','rhigh'])
    
    #市值和BM双重排序
    small_low4 = df_daily.query('(label_sb4 == "small") & (label_bm4 == "low")')
    small_mid4 = df_daily.query('(label_sb4 == "small") & (label_bm4 == "mid")')
    small_high4 = df_daily.query('(label_sb4 == "small") & (label_bm4 == "high")')
    
    large_low4 = df_daily.query('(label_sb4 == "large") & (label_bm4 == "low")')
    large_mid4 = df_daily.query('(label_sb4 == "large") & (label_bm4 == "mid")')
    large_high4 = df_daily.query('(label_sb4 == "large") & (label_bm4 == "high")')
    
    r_sl4 = (small_low4['month_return']*small_low4['MarketValue']).sum()/small_low4['MarketValue'].sum()
    r_sm4 = (small_mid4['month_return']*small_mid4['MarketValue']).sum()/small_mid4['MarketValue'].sum()
    r_sh4 = (small_high4['month_return']*small_high4['MarketValue']).sum()/small_high4['MarketValue'].sum()
    
    r_ll4 = (large_low4['month_return']*large_low4['MarketValue']).sum()/large_low4['MarketValue'].sum()
    r_lm4 = (large_mid4['month_return']*large_mid4['MarketValue']).sum()/large_mid4['MarketValue'].sum()
    r_lh4 = (large_high4['month_return']*large_high4['MarketValue']).sum()/large_high4['MarketValue'].sum()
    
    #市值和ROA双重排序
    small_rlow4 = df_daily.query('(label_sb4 == "small") & (label_roa4 == "rlow")')
    small_rmid4 = df_daily.query('(label_sb4 == "small") & (label_roa4 == "rmid")')
    small_rhigh4 = df_daily.query('(label_sb4 == "small") & (label_roa4 == "rhigh")')
    
    large_rlow4 = df_daily.query('(label_sb4 == "large") & (label_roa4 == "rlow")')
    large_rmid4 = df_daily.query('(label_sb4 == "large") & (label_roa4 == "rmid")')
    large_rhigh4 = df_daily.query('(label_sb4 == "large") & (label_roa4 == "rhigh")')
    
    r_srl4 = (small_rlow4['month_return']*small_rlow4['MarketValue']).sum()/small_rlow4['MarketValue'].sum()
    r_srm4 = (small_rmid4['month_return']*small_rmid4['MarketValue']).sum()/small_rmid4['MarketValue'].sum()
    r_srh4 = (small_rhigh4['month_return']*small_rhigh4['MarketValue']).sum()/small_rhigh4['MarketValue'].sum()
    
    r_lrl4 = (large_rlow4['month_return']*large_rlow4['MarketValue']).sum()/large_rlow4['MarketValue'].sum()
    r_lrm4 = (large_rmid4['month_return']*large_rmid4['MarketValue']).sum()/large_rmid4['MarketValue'].sum()
    r_lrh4 = (large_rhigh4['month_return']*large_rhigh4['MarketValue']).sum()/large_rhigh4['MarketValue'].sum()    
    
    #计算因子收益
    liu4_smb_one = (r_sl4+r_sm4+r_sh4-r_ll4-r_lm4-r_lh4)/3 
    liu4_smb_two = (r_srl4+r_srm4+r_srh4-r_lrl4-r_lrm4-r_lrh4)/3
    liu4_smb = ( (r_sl4+r_sm4+r_sh4-r_ll4-r_lm4-r_lh4)/3 + (r_srl4+r_srm4+r_srh4-r_lrl4-r_lrm4-r_lrh4)/3 ) /2
    liu4_hml = (r_lh4+r_sh4-r_ll4-r_sl4)/2
    liu4_rmw = (r_lrh4+r_srh4-r_lrl4-r_srl4)/2
    
    return liu4_smb,liu4_hml,liu4_rmw

In [ ]:
#计算Liu ，Stambaugh and Yuan（2019）三因子
liu3_data = []
for date,group in fd_trade_month.groupby('Trddt'):
    liu3_smb, liu3_hml = liu3(group)
    liu3_data.append([date, liu3_smb, liu3_hml])

In [ ]:
#计算Liu，Shi and Lian（2019）四因子
liu4_data = []
for date,group in fd_trade_month.groupby('Trddt'):
    liu4_smb, liu4_hml, liu4_rmw = liu4(group)
    liu4_data.append([date, liu4_smb, liu4_hml, liu4_rmw])

In [ ]:
df_liu3 = pd.DataFrame(np.array(liu3_data),columns=['date','liu3_smb', 'liu3_hml'])
df_liu4 = pd.DataFrame(np.array(liu4_data),columns = ['date', 'liu4_smb', 'liu4_hml', 'liu4_rmw'])
df_factor = pd.merge(df_liu3,df_liu4,how='inner',on='date')
df_factor = df_factor.set_index(df_factor['date'])
del df_factor['date']
df_factor

In [ ]:
#画出因子的累计收益率
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(2,2,1)
ax1.plot((df_factor['liu3_smb']+1).cumprod())
ax1.set_xlabel('SMB')
ax2 = fig.add_subplot(2,2,2)
ax2.plot((df_factor['liu3_hml']+1).cumprod())
ax2.set_xlabel('HML(EP)')
ax3 = fig.add_subplot(2,2,3)
ax3.plot((df_factor['liu4_hml']+1).cumprod())
ax3.set_xlabel('HML(BM)')
ax4 = fig.add_subplot(2,2,4)
ax4.plot((df_factor['liu4_rmw']+1).cumprod())
ax4.set_xlabel('RMW')
plt.subplots_adjust(wspace=0.2,hspace=0.3)
plt.show()

In [ ]:
#调用科学计算包
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用异象检验Liu ，Stambaugh and Yuan（2019）三因子模型效率(以投资和动量为例)
df_test = pd.merge(df_factor,fd_dl_inv,how='inner',on='date')
df_test = df_test.astype(float) #将所有数据类型转换为浮点型
np.seterr(divide='ignore',invalid='ignore')#去除不可逆矩阵
winsorize(df_test.all(),limits=[0.01, 0.01]) #对所有列进行上下1%的缩尾处理
result_one = smf.ols(formula = 'CMA1 ~  RiskPremium1 + liu3_smb + liu3_hml', data=df_test).fit()
result_one.summary()

In [ ]:
result_two = smf.ols(formula = 'UMD1 ~  RiskPremium1 + liu3_smb + liu3_hml', data=df_test).fit()
result_two.summary()

In [ ]:
#使用异象检验Liu，Shi and Lian（2019）四因子模型效率(以投资和动量为例)
model_three = sm.OLS(df_test['CMA1'], sm.add_constant(
        df_test[['RiskPremium1','liu4_smb','liu4_hml','liu4_rmw']].values))
result_three = model_three.fit()
print(result_three.summary())

In [ ]:
model_four = sm.OLS(df_test['UMD1'], sm.add_constant(
        df_test[['RiskPremium1','liu4_smb','liu4_hml','liu4_rmw']].values))
result_four = model_four.fit()
print(result_four.summary())

In [ ]:
#画出因子的累计收益率
fig = plt.figure(figsize=(15,6))
ax5 = fig.add_subplot(1,2,1)
ax5.plot((df_test['UMD1']+1).cumprod())
ax5.set_xlabel('UMD1')
ax6 = fig.add_subplot(1,2,2)
ax6.plot((df_test['CMA1']+1).cumprod())
ax6.set_xlabel('CMA1')

In [ ]:
#模型之间的对比（GRS检验）
#先使用liu3_smb和liu3_hml作为测试资产
#计算残差待用
result_liu3_smb = smf.ols(formula = 'liu3_smb ~ RiskPremium1 + liu4_smb + liu4_hml + liu4_rmw' ,data = df_test).fit()
result_liu3_hml = smf.ols(formula = 'liu3_hml ~ RiskPremium1 + liu4_smb + liu4_hml + liu4_rmw' ,data = df_test).fit()
res_liu3 = np.array([result_liu3_smb.resid.values , result_liu3_hml.resid.values])#将两个数组在列上合并
res_liu3 = res_liu3.T #转置获得T*2的数组
alpha3 = np.array([[result_liu3_smb.params[0]],[result_liu3_hml.params[0]]]) #获得2*1的定价误差数组
factor3 = np.array([df_test['RiskPremium1'].values ,df_test['liu4_smb'].values,df_test['liu4_hml'].values,df_test['liu4_rmw'].values])
factor3 = factor3.T

In [ ]:
#调用线性代数相关包
import scipy
from numpy.linalg import inv
def GRS(alpha, resids, factor):
    # GRS test statistic
    # N个待检验资产组合, L个因子, and T 时间周期
    # alpha是定价误差，Nx1的数组
    # resids回归的残差，TxN的数组
    # factor是因子收益率，TxL的数组
    T, N = resids.shape
    L = factor.shape[1]
    factor_mean = np.nanmean(factor, axis=0)
    cov_resids = np.matmul(resids.T, resids) / (T-L-1)
    cov_fac = np.matmul(np.array(factor - np.nanmean(factor, axis=0)).T, np.array(factor - np.nanmean(factor, axis=0))) / T-1
    GRS = (T / N) * ((T - N - L) / (T - L - 1)) * ((np.matmul(np.matmul(alpha.T, inv(cov_resids)), alpha)) / (1 + (np.matmul(np.matmul(factor_mean.T, inv(cov_fac)), factor_mean))))
    pVal = 1 - scipy.stats.f.cdf(GRS, N, T - N - L) #需引用F分布“scipy.stats.f”
    return GRS, pVal

In [ ]:
GRS_liu3 , Pval_liu3 = GRS(alpha3, res_liu3, factor3)
Pval_liu3

In [ ]:
#再使用liu4_smb、liu4_hml和liu4_rmw作为测试资产
result_liu4_smb = smf.ols(formula = 'liu4_smb ~ RiskPremium1 + liu3_smb + liu3_hml' ,data = df_test).fit()
result_liu4_hml = smf.ols(formula = 'liu4_hml ~ RiskPremium1 + liu3_smb + liu3_hml' ,data = df_test).fit()
result_liu4_rmw = smf.ols(formula = 'liu4_rmw ~ RiskPremium1 + liu3_smb + liu3_hml' ,data = df_test).fit()
res_liu4 = np.array([result_liu4_smb.resid.values , result_liu4_hml.resid.values , result_liu4_rmw.resid.values])#将两个数组在列上合并
res_liu4 = res_liu4.T #转置获得T*2的数组
alpha4 = np.array([[result_liu4_smb.params[0]],[result_liu4_hml.params[0]],[result_liu4_rmw.params[0]]]) #获得2*1的定价误差数组
factor4 = np.array([df_test['liu4_smb'].values,df_test['liu4_hml'].values,df_test['liu4_rmw'].values])
factor4 = factor4.T

In [ ]:
GRS_liu4 , Pval_liu4 = GRS(alpha4, res_liu4, factor4)
Pval_liu4